In [ ]:
import pandas as pd
from collections import defaultdict

# Load data
df = pd.read_csv(r"C:\Users\stdFurqan\Desktop\paft\pos\train.csv")

# Preserve tag order
unique_tags = df['tag'].unique().tolist()

# Label mapping
label_map = {tag: idx for idx, tag in enumerate(unique_tags)}
classes = unique_tags.copy()

# Collect UNIQUE words per tag (order preserved)
tag_words = defaultdict(list)

for _, row in df.iterrows():
    tag = row['tag']
    word = str(row['word']).strip()

    if word not in tag_words[tag]:
        tag_words[tag].append(word)

# Generate InputExamples
print("### InputExamples ###\n")

guid = 0
tag_stats = {}  # to track how many examples per tag

for tag in unique_tags:
    label = label_map[tag]

    # Take min(16, available unique words)
    selected_words = tag_words[tag][:16]
    tag_stats[tag] = len(selected_words)

    for word in selected_words:
        print(f'InputExample(guid={guid}, text_a="{word}", label={label}),')
        guid += 1

# Print metadata
print("\n### Classes ###")
print(f"classes = {classes}")

print("\n### Label Map ###")
print(f"label_map = {label_map}")

# Optional: show summary
print("\n### Samples per Tag ###")
for tag, count in tag_stats.items():
    print(f"{tag}: {count}")

print(f"\nTotal InputExamples generated: {guid}")


In [ ]:
from openprompt.prompts import ManualTemplate

templates = [
    ("P1", ManualTemplate(
        text='لفظ: {"placeholder":"text_a"} → حصہ کلام: {"mask"}',
        tokenizer=tokenizer,
    )),
    ("P2", ManualTemplate(
        text='جملے میں یہ لفظ {"placeholder":"text_a"} کس قسم کا ہے: {"mask"}۔',
        tokenizer=tokenizer,
    )),
    ("P3", ManualTemplate(
        text='لفظ {"placeholder":"text_a"} کا POS tag کیا ہے؟ {"mask"}',
        tokenizer=tokenizer,
    )),
    ("P4", ManualTemplate(
        text='یہ لفظ {"placeholder":"text_a"} جملے میں کس زمرے کا ہے؟ {"mask"}',
        tokenizer=tokenizer,
    )),
    ("P5", ManualTemplate(
        text='{"placeholder":"text_a"} → حصہ کلام: {"mask"}',
        tokenizer=tokenizer,
    )),
    ("P6", ManualTemplate(
        text='لفظ: {"placeholder":"text_a"} اس کا حصہ کلام {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),
    ("P7", ManualTemplate(
        text='POS tag for {"placeholder":"text_a"} is {"mask"}',
        tokenizer=tokenizer,
    )),
    ("P8", ManualTemplate(
        text='{"placeholder":"text_a"} کس قسم کا لفظ ہے: {"mask"}',
        tokenizer=tokenizer,
    )),
    ("P9", ManualTemplate(
        text='لفظ {"placeholder":"text_a"} → POS: {"mask"}',
        tokenizer=tokenizer,
    )),
    ("P10", ManualTemplate(
        text='جملے میں لفظ {"placeholder":"text_a"} کس POS category سے تعلق رکھتا ہے؟ {"mask"}',
        tokenizer=tokenizer,
    )),
]


In [ ]:
import pandas as pd
from openprompt.data_utils import InputExample

# Load your dataset
df = pd.read_csv(r"C:\Users\stdFurqan\Desktop\paft\pos\train.csv")  # columns: 'word', 'tag'

# Find unique tags
unique_tags = df['tag'].unique()
print("Unique POS tags:", unique_tags)

# Create label map
label_map = {tag: i for i, tag in enumerate(unique_tags)}
print("Label map:", label_map)

# Prepare few-shot dataset: first 16 examples per tag
train_dataset = []
guid = 0

for tag in unique_tags:
    # Filter rows with this tag
    tag_rows = df[df['tag'] == tag].head(16)  # first 16 examples
    for _, row in tag_rows.iterrows():
        example = InputExample(
            guid=str(guid),
            text_a=row['word'],
            label=label_map[row['tag']]
        )
        train_dataset.append(example)
        guid += 1

print("✅ Few-shot dataset created with total examples:", len(train_dataset))

# Inspect first few examples
for ex in train_dataset[:100]:
    print("GUID:", ex.guid, "| Word:", ex.text_a, "| Label:", ex.label)


In [ ]:
import os
import shutil

# Get user profile path
user_profile = os.environ["USERPROFILE"]

# Paths to Hugging Face cached models
cached_models = [
    os.path.join(user_profile, r".cache\huggingface\hub\models--bert-base-multilingual-cased"),
    os.path.join(user_profile, r".cache\huggingface\hub\models--xlm-roberta-base"),
    os.path.join(user_profile, r".cache\huggingface\hub\models--roberta-base")
]

# Remove cached models if they exist
for path in cached_models:
    if os.path.exists(path):
        shutil.rmtree(path)
        print(f"Removed cache: {path}")
    else:
        print(f"No cache found at: {path}")


In [1]:
# Set device
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
# ==============================
# Imports
# ==============================
import torch
import pandas as pd
import random
import numpy as np
from collections import Counter
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
from openprompt.prompts import ManualTemplate, ManualVerbalizer
from openprompt.data_utils import InputExample
from openprompt.plms import load_plm
from openprompt import PromptForClassification, PromptDataLoader
from torch.optim import AdamW
from sklearn.metrics import classification_report
from collections import defaultdict
from torch.utils.data import DataLoader, Sampler

# ========================================
# Check CUDA
# ========================================
device = "cuda" #if torch.cuda.is_available() else "cpu"
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
    print("CUDA version:", torch.version.cuda)
    print("GPU count:", torch.cuda.device_count())

# ========================================
# Seeds for reproducibility
# ========================================
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: True
GPU name: NVIDIA GeForce RTX 4080 SUPER
CUDA version: 12.1
GPU count: 1


In [3]:
# ==============================
# Balanced Batch Sampler
# ==============================
class BalancedBatchSampler(Sampler):
    def __init__(self, dataset, batch_size):
        """
        dataset: list of InputExample
        batch_size: total batch size (must be divisible by number of classes)
        """
        self.dataset = dataset
        self.labels = [ex.label for ex in dataset]
        self.classes = list(sorted(set(self.labels)))
        self.num_classes = len(self.classes)
        assert batch_size % self.num_classes == 0, "Batch size must be divisible by number of classes"
        self.batch_size_per_class = batch_size // self.num_classes

    def __iter__(self):
        class_indices = {c: np.where(np.array(self.labels) == c)[0].tolist() for c in self.classes}
        for c in self.classes:
            np.random.shuffle(class_indices[c])

        num_batches = min(len(class_indices[c]) // self.batch_size_per_class for c in self.classes)

        for i in range(num_batches):
            batch = []
            for c in self.classes:
                start = i * self.batch_size_per_class
                end = start + self.batch_size_per_class
                batch.extend(class_indices[c][start:end])
            np.random.shuffle(batch)
            yield batch

    def __len__(self):
        return min(len(np.where(np.array(self.labels) == c)[0]) // self.batch_size_per_class for c in self.classes)

In [4]:


# ==============================
# Load POS Train Dataset
# ==============================
df = pd.read_csv(r"C:\Users\stdFurqan\Desktop\paft\pos\train.csv")  # columns: 'word', 'tag'

# Find unique tags
unique_tags = df['tag'].unique()
print("Unique POS tags:", unique_tags)

# Create label map
label_map = {tag: i for i, tag in enumerate(unique_tags)}
print("Label map:", label_map)

# ==============================
# Prepare Few-Shot Training Dataset: first 16 examples per tag
# ==============================
train_dataset = []
guid = 0

for tag in unique_tags:
    tag_rows = df[df['tag'] == tag].head(16)  # first 16 examples
    for _, row in tag_rows.iterrows():
        example = InputExample(
            guid=str(guid),
            text_a=row['word'],
            label=label_map[row['tag']]
        )
        train_dataset.append(example)
        guid += 1

print("✅ Few-shot dataset created with total examples:", len(train_dataset))

# ==============================
# Inspect first few examples
# ==============================
for ex in train_dataset[:20]:
    print("GUID:", ex.guid, "| Word:", ex.text_a, "| Label:", ex.label)

# ==============================
# Define Classes
# ==============================
classes = list(label_map.keys())
print("Classes:", classes)

# ==============================
# Load Pretrained Model and Tokenizer
# ==============================

# # # Step 1: Use load_plm with 'roberta' to get the correct WrapperClass
# _, _, _, WrapperClass = load_plm("roberta", "roberta-base")  # Just to get the wrapper

# # # Step 2: Manually load XLM-RoBERTa model/tokenizer
# model_name = "xlm-roberta-base"
# tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
# plm = XLMRobertaForMaskedLM.from_pretrained(model_name)

# ==============================
# Load Pretrained Language Model (mBERT)
# ==============================
plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-multilingual-cased")

Unique POS tags: ['PN' 'G' 'NN' 'P' 'U' 'VB' 'SM' 'PM' 'PP' 'CC' 'ADJ' 'CA' 'RP' 'SC' 'SE'
 'ADV' 'EXP' 'I' 'NEG' 'TA' 'AP' 'Q' 'PD' 'WALA' 'KP' 'GR' 'REP' 'A' 'KD'
 'AA' 'QW' 'KER' 'OR' 'AKP' 'MUL' 'INT' 'AD' 'FR' 'DATE' 'RD']
Label map: {'PN': 0, 'G': 1, 'NN': 2, 'P': 3, 'U': 4, 'VB': 5, 'SM': 6, 'PM': 7, 'PP': 8, 'CC': 9, 'ADJ': 10, 'CA': 11, 'RP': 12, 'SC': 13, 'SE': 14, 'ADV': 15, 'EXP': 16, 'I': 17, 'NEG': 18, 'TA': 19, 'AP': 20, 'Q': 21, 'PD': 22, 'WALA': 23, 'KP': 24, 'GR': 25, 'REP': 26, 'A': 27, 'KD': 28, 'AA': 29, 'QW': 30, 'KER': 31, 'OR': 32, 'AKP': 33, 'MUL': 34, 'INT': 35, 'AD': 36, 'FR': 37, 'DATE': 38, 'RD': 39}
✅ Few-shot dataset created with total examples: 627
GUID: 0 | Word: ’ | Label: 0
GUID: 1 | Word: ای | Label: 0
GUID: 2 | Word: بلال | Label: 0
GUID: 3 | Word: بھائی | Label: 0
GUID: 4 | Word: ستمبر | Label: 0
GUID: 5 | Word: اڑھائی | Label: 0
GUID: 6 | Word: “ | Label: 0
GUID: 7 | Word: ٹیگ | Label: 0
GUID: 8 | Word: رو | Label: 0
GUID: 9 | Word: مین | Label: 0

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized fr

In [5]:
# ==============================
# Define Prompt Template
# ==============================
template = ManualTemplate(
    text='لفظ: {"placeholder":"text_a"} → حصہ کلام: {"mask"}',
    tokenizer=tokenizer,
)

templates = [
    ("P1", ManualTemplate(
        text='لفظ: {"placeholder":"text_a"} → حصہ کلام: {"mask"}',
        tokenizer=tokenizer,
    )),
    ("P2", ManualTemplate(
        text='جملے میں یہ لفظ {"placeholder":"text_a"} کس قسم کا ہے: {"mask"}۔',
        tokenizer=tokenizer,
    )),
    ("P3", ManualTemplate(
        text='لفظ {"placeholder":"text_a"} کا POS tag کیا ہے؟ {"mask"}',
        tokenizer=tokenizer,
    )),
    ("P4", ManualTemplate(
        text='یہ لفظ {"placeholder":"text_a"} جملے میں کس زمرے کا ہے؟ {"mask"}',
        tokenizer=tokenizer,
    )),
    ("P5", ManualTemplate(
        text='{"placeholder":"text_a"} → حصہ کلام: {"mask"}',
        tokenizer=tokenizer,
    )),
    ("P6", ManualTemplate(
        text='لفظ: {"placeholder":"text_a"} اس کا حصہ کلام {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),
    ("P7", ManualTemplate(
        text='POS tag for {"placeholder":"text_a"} is {"mask"}',
        tokenizer=tokenizer,
    )),
    ("P8", ManualTemplate(
        text='{"placeholder":"text_a"} کس قسم کا لفظ ہے: {"mask"}',
        tokenizer=tokenizer,
    )),
    ("P9", ManualTemplate(
        text='لفظ {"placeholder":"text_a"} → POS: {"mask"}',
        tokenizer=tokenizer,
    )),
    ("P10", ManualTemplate(
        text='جملے میں لفظ {"placeholder":"text_a"} کس POS category سے تعلق رکھتا ہے؟ {"mask"}',
        tokenizer=tokenizer,
    )),
]

# Group words by POS tag
tag_to_words = defaultdict(list)

for _, row in df.iterrows():
    word = str(row["word"]).strip()
    tag = row["tag"]
    if word and word not in tag_to_words[tag]:
        tag_to_words[tag].append(word)

# Build verbalizer map
label_words_map = {}

for tag in classes:
    examples = tag_to_words.get(tag, [])

    # Take first 2 words if available
    sample_words = examples[:2]

    # Always include the tag itself
    label_words_map[tag] = [tag] + sample_words


verbalizer = ManualVerbalizer(
    classes=classes,
    label_words=label_words_map,
    tokenizer=tokenizer
)



print("\n📘 FINAL VERBALIZER CONTENT:\n")

for tag, words in label_words_map.items():
    print(f"{tag} → {words}")

 

# # ==============================
# # Create Prompt Model
# # ==============================
# prompt_model = PromptForClassification(
#     template=template,
#     plm=plm,
#     verbalizer=verbalizer
# )
# prompt_model = prompt_model.to(device)
# # prompt_model.to(device)




📘 FINAL VERBALIZER CONTENT:

PN → ['PN', '’', 'ای']
G → ['G', 'میرے', 'میرا']
NN → ['NN', 'بھائی', 'محنت']
P → ['P', 'کا', 'کی']
U → ['U', 'میل', 'من']
VB → ['VB', 'آیاہے', 'ہیں']
SM → ['SM', '۔', '!']
PM → ['PM', '،', '"']
PP → ['PP', 'ہم', 'آپ']
CC → ['CC', 'اور', 'و']
ADJ → ['ADJ', 'قائل', 'میریئٹ']
CA → ['CA', 'اکیس', 'دو']
RP → ['RP', 'خود', 'اپنےآپ']
SC → ['SC', 'جبکہ', 'کہ']
SE → ['SE', 'سے']
ADV → ['ADV', 'برائے', 'زیادہ']
EXP → ['EXP', '”', '(']
I → ['I', 'تو', 'ہی']
NEG → ['NEG', 'نہیں', 'نہ']
TA → ['TA', 'ہیں', 'ہے']
AP → ['AP', 'وہاں', 'اب']
Q → ['Q', 'کچھ', 'ہر']
PD → ['PD', 'اس', 'ان']
WALA → ['WALA', 'والا', 'والے']
KP → ['KP', 'کس', 'کیا']
GR → ['GR', 'اپنی', 'اپنا']
REP → ['REP', 'جس', 'جن']
A → ['A', 'سی', 'سا']
KD → ['KD', 'کسی', 'کس']
AA → ['AA', 'سکتے', 'گیا']
QW → ['QW', 'کیا', 'کیوں']
KER → ['KER', 'کر']
OR → ['OR', 'دونوں', 'ساتویں']
AKP → ['AKP', 'کہیں', 'کیسے']
MUL → ['MUL', 'شاپنگ', 'افطار']
INT → ['INT', 'واہ', 'القادری']
AD → ['AD', 'ایسی']
FR → ['FR', 'آد

In [6]:
# ==============================
# Create Prompt Model
# ==============================
prompt_model = PromptForClassification(
    template=template,
    plm=plm,
    verbalizer=verbalizer
)
# prompt_model.to(device)

# ==============================
# Training loop with BalancedBatchSampler + random template switching
# ==============================
T = 20   # epochs
K = 1    # steps per prompt
batch_size = 157

prompt_model.train()
optimizer = AdamW(prompt_model.parameters(), lr=1e-5)
all_epoch_patterns = {}

# for epoch in range(T):
#     print(f"\n🟦 Epoch {epoch+1}/{T}")

#     # Random initial template
#     prompt_name, current_template = random.choice(templates)
#     epoch_pattern = []

#     # Create PromptDataLoader with BalancedBatchSampler
#     sampler = BalancedBatchSampler(train_dataset, batch_size=batch_size)
#     train_loader = PromptDataLoader(
#         dataset=train_dataset,
#         tokenizer=tokenizer,
#         template=current_template,
#         tokenizer_wrapper_class=WrapperClass,
#         max_seq_length=128,
#         batch_size=batch_size,
#         batch_sampler=sampler,
#         shuffle=False  # shuffle is ignored when batch_sampler is used
#     )

#     step_counter = 0
#     epoch_loss = 0.0

#     for batch in train_loader:
#         # Move batch to device
#         # batch = {k: v.to(device) if torch.is_tensor(v) else v for k, v in batch.items()}
    
#         optimizer.zero_grad()
#         logits = prompt_model(batch)
#         loss = torch.nn.CrossEntropyLoss()(logits, batch['label'])
#         loss.backward()
#         optimizer.step()

#         epoch_loss += loss.item()
#         epoch_pattern.append(prompt_name)

#         step_counter += 1

#         # Switch template every K steps
#         if step_counter % K == 0:
#             prompt_name, current_template = random.choice(templates)

#             # Rebuild PromptDataLoader with new template but same sampler
#             train_loader = PromptDataLoader(
#                 dataset=train_dataset,
#                 tokenizer=tokenizer,
#                 template=current_template,
#                 tokenizer_wrapper_class=WrapperClass,
#                 max_seq_length=128,
#                 batch_size=batch_size,
#                 batch_sampler=sampler,
#                 shuffle=False
#             )

#     all_epoch_patterns[f"epoch_{epoch+1}"] = epoch_pattern
#     print(f"Epoch {epoch+1} Loss: {epoch_loss:.4f}")
#     print(f"Prompt pattern: {epoch_pattern}")



for epoch in range(T):
    print(f"\n🟦 Epoch {epoch+1}/{T}")

    # Random initial prompt
    prompt_name, current_template = random.choice(templates)

    step_counter = 0
    epoch_loss = 0.0

    # Record prompt usage pattern
    epoch_pattern = []

    train_loader = PromptDataLoader(
        dataset=train_dataset,
        tokenizer=tokenizer,
        template=current_template,
        tokenizer_wrapper_class=WrapperClass,
        max_seq_length=128,
        batch_size=batch_size,
        shuffle=True
    )

    for batch in train_loader:
        optimizer.zero_grad()

        logits = prompt_model(batch)
        loss = torch.nn.CrossEntropyLoss()(logits, batch['label'])

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        # Log which prompt was used at this step
        epoch_pattern.append(prompt_name)

        step_counter += 1

        # Switch prompt every K steps
        if step_counter % K == 0:
            prompt_name, current_template = random.choice(templates)

            train_loader = PromptDataLoader(
                dataset=train_dataset,
                tokenizer=tokenizer,
                template=current_template,
                tokenizer_wrapper_class=WrapperClass,
                max_seq_length=128,
                batch_size=batch_size,
                shuffle=True
            )

    all_epoch_patterns[f"epoch_{epoch+1}"] = epoch_pattern

    print(f"Epoch {epoch+1} Loss: {epoch_loss:.4f}")
    print(f"Prompt pattern: {epoch_pattern}")



🟦 Epoch 1/20


tokenizing: 627it [00:00, 3930.51it/s]
tokenizing: 627it [00:00, 3460.71it/s]
tokenizing: 627it [00:00, 4061.10it/s]
tokenizing: 627it [00:00, 3017.90it/s]
tokenizing: 627it [00:00, 3095.16it/s]


Epoch 1 Loss: 14.3387
Prompt pattern: ['P2', 'P1', 'P5', 'P4']

🟦 Epoch 2/20


tokenizing: 627it [00:00, 4376.91it/s]
tokenizing: 627it [00:00, 2823.66it/s]
tokenizing: 627it [00:00, 4118.42it/s]
tokenizing: 627it [00:00, 2821.93it/s]
tokenizing: 627it [00:00, 2662.48it/s]


Epoch 2 Loss: 13.3603
Prompt pattern: ['P3', 'P2', 'P9', 'P2']

🟦 Epoch 3/20


...

Epoch 18 Loss: 2.9344
Prompt pattern: ['P1', 'P6', 'P7', 'P5']

🟦 Epoch 19/20


tokenizing: 627it [00:00, 4021.16it/s]
tokenizing: 627it [00:00, 2680.85it/s]
tokenizing: 627it [00:00, 4300.08it/s]
tokenizing: 627it [00:00, 4023.46it/s]
tokenizing: 627it [00:00, 5442.95it/s]


Epoch 19 Loss: 2.3602
Prompt pattern: ['P4', 'P10', 'P6', 'P4']

🟦 Epoch 20/20


tokenizing: 627it [00:00, 3888.69it/s]
tokenizing: 627it [00:00, 4008.83it/s]
tokenizing: 627it [00:00, 1434.41it/s]
tokenizing: 627it [00:00, 5165.18it/s]
tokenizing: 627it [00:00, 3613.75it/s]

Epoch 20 Loss: 3.3284
Prompt pattern: ['P7', 'P8', 'P3', 'P5']


In [7]:

# ==============================
# Prepare Evaluation Dataset
# ==============================
df_eval = pd.read_csv(r"C:\Users\stdFurqan\Desktop\paft\pos\test.csv")  # columns: 'word', 'tag'
eval_dataset = [
    InputExample(guid=str(i), text_a=row['word'], label=label_map[row['tag']])
    for i, row in df_eval.iterrows()
]

# ==============================
# DataLoader for Evaluation
# ==============================
eval_loader = PromptDataLoader(
    dataset=eval_dataset,
    tokenizer=tokenizer,
    template=template,
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=8,
    shuffle=False
)

# ==============================
# Evaluate Model
# ==============================
prompt_model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in eval_loader:
        # Move tensors to device
        # batch = {k: v.to(device) if torch.is_tensor(v) else v for k, v in batch.items()}
        logits = prompt_model(batch)
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(batch['label'].cpu().tolist())

# ==============================
# Classification Report
# ==============================
print("\n📊 POS Tagging Classification Report:")
# print(classification_report(all_labels, all_preds, target_names=classes, digits=4))
# ==============================
# Classification Report (FIXED)
# ==============================

all_label_ids = list(range(len(classes)))  # [0, 1, 2, ..., 39]

print("\n📊 POS Tagging Classification Report:")
print(
    classification_report(
        all_labels,
        all_preds,
        labels=all_label_ids,      # 👈 IMPORTANT FIX
        target_names=classes,
        digits=4,
        zero_division=0
    )
)


tokenizing: 86676it [00:19, 4510.94it/s]



📊 POS Tagging Classification Report:

📊 POS Tagging Classification Report:
              precision    recall  f1-score   support

          PN     0.2514    0.1484    0.1866      6065
           G     0.4085    0.9093    0.5638       474
          NN     0.5827    0.1459    0.2334     21792
           P     0.9362    0.8859    0.9104     10256
           U     0.0285    0.8500    0.0552        40
          VB     0.5777    0.1079    0.1818     10060
          SM     0.9988    0.9281    0.9621      3464
          PM     0.9940    0.9496    0.9713      1924
          PP     0.6236    0.4511    0.5235      3316
          CC     0.9585    0.9066    0.9318      1938
         ADJ     0.1328    0.1417    0.1371      5342
          CA     0.3013    0.6551    0.4127      1763
          RP     0.8830    0.9881    0.9326        84
          SC     0.7899    0.5811    0.6696      2504
          SE     0.8659    1.0000    0.9281      1440
         ADV     0.2285    0.6541    0.3386      1480
     